# Analyze sequences in H3 2023-2024 `seqneut-library` and produce plots

Author: Caroline Kikawa

In [1]:
# Import packages
import os
import numpy as np
import pandas as pd
import altair as alt
from Bio import SeqIO


# Ignore error message from Altair about large dataframes
_ = alt.data_transformers.disable_max_rows()

color_scheme = [
    '#345995', #blue
    '#03cea4', #teal
    '#ca1551', #red
    '#eac435', #yellow
               ]



In [2]:
# Identify input and output
resultsdir = '../results'
datadir = '../data'
os.makedirs(resultsdir, exist_ok=True)
os.makedirs(datadir, exist_ok=True)

protein_sequences = [
    '2023-2024_H3_library_protein_HA_ectodomain.fasta', # Entire HA ectodomain
    '2023-2024_H3_library_protein_HA1.fasta' # Just HA1 sequences
]

# Define virus order
viral_plot_orderfile = '../../../data/H3N2library_2023-2024_strain_order.csv'
viral_plot_order = pd.read_csv(viral_plot_orderfile)
viruses = [v for v in viral_plot_order.strain]

# Define vaccine strains
vaccine_strains = []
with open('../../plot_NT50_profiles/data/vaccine_strains.csv') as f:
    for line in f:
        line = line.strip('\n')
        if 'strain' not in line:
            vaccine_strains.append(line)

# Currently circulating strains list
library_strains = ([x for x in viruses if x not in vaccine_strains])
library_strains.append('A/Massachusetts/18/2022') # Massachusetts/18 is technically in both

In [3]:
# Define strain metadata
strain_metadatafile = os.path.join(datadir, 'strain_metadata.csv')
strain_metadata = (pd.read_csv(strain_metadatafile, sep = ';')
                   .drop(columns = 'index')
                   .rename(columns = {'seqName': 'strain'}))

# Replace values in strain metadata with strings matching the plotted strain names
virus_dict = {'A/Texas/50/2012_X-223A_13/252': 'A/Texas/50/2012_X-223A_(13/252)',
              'A/Hong_Kong/4801/2014_15/192': 'A/Hong_Kong/4801/2014_(15/192)'}

strain_metadata['strain'] = strain_metadata['strain'].replace(virus_dict)

## Calculate Hamming distances between sequences
First define a bunch of utility functions for producing Hamming distance matrices.

In [4]:
# Function to calculate Hamming distance between two sequences
def hamming_distance(seq1, seq2, region=None):
    if region is None:  # If no region is specified, compare the whole sequence
        return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))
    else:
        # Compare only specified region
        start, end = region
        return sum(c1 != c2 for c1, c2 in zip(seq1[start:end], seq2[start:end]))

# Load protein sequences and their names from FASTA file
def load_fasta_sequences(fasta_file):
    sequences = []
    names = []
    for record in SeqIO.parse(fasta_file, "fasta"):
        sequences.append(str(record.seq))
        names.append(record.id)  # Use record.id for sequence name (header)
    return sequences, names

# Compute pairwise Hamming distances
def compute_hamming_distances(sequences):
    n = len(sequences)
    dist_matrix = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            dist = hamming_distance(sequences[i], sequences[j])
            dist_matrix[i, j] = dist
            dist_matrix[j, i] = dist
            
    return dist_matrix

Now we want to produce some triangular heatmaps to look at this data more easily. 

In [5]:
def plot_triangular_hamming_distances(fasta_file, sequences_to_plot, ordered_names):
    # Configure
    fontSize = 18

    region = fasta_file.split('.fasta')[0].split('_')[-1]
    
    # Load sequences and their names from FASTA
    sequences, names = load_fasta_sequences(fasta_file)

    # Check if sequences are of the same length, which is required for Hamming distance
    seq_len = set(len(seq) for seq in sequences)
    if len(seq_len) != 1:
        raise ValueError("All sequences must be of the same length to compute Hamming distances.")

    # Compute pairwise Hamming distances
    dist_matrix = compute_hamming_distances(sequences)
    
    # Create a DataFrame for the distance matrix using sequence names
    df = pd.DataFrame(dist_matrix, columns=names, index=names)

    # If a subset of sequences is provided, filter the distance matrix
    if sequences_to_plot is not None:
        df = df.loc[sequences_to_plot, sequences_to_plot]

    # Melt the DataFrame to long format for Altair plotting
    df_melt = df.reset_index().melt(id_vars='index')
    df_melt.columns = ['Seq1', 'Seq2', 'Hamming Distance']

    # If a custom order is provided, use it to order the axes, keeping only the names in sequences_to_plot
    if ordered_names is not None:
        ordered_names_filtered = [name for name in ordered_names if name in df.columns]
    else:
        ordered_names_filtered = df.columns.tolist()

    # Create a dictionary to map sequence names to their order
    order_dict = {name: idx for idx, name in enumerate(ordered_names_filtered)}

    # Filter for the upper triangle: Keep only rows where Seq1 appears earlier than Seq2 in the order
    df_melt = df_melt[df_melt['Seq1'].map(order_dict) < df_melt['Seq2'].map(order_dict)]

    # Plot with Altair
    heatmap = alt.Chart(df_melt
                       ).mark_rect(
        # cornerRadius=10,
        strokeWidth=0
                       ).encode(
        x=alt.X('Seq1:O', sort=ordered_names_filtered, axis=alt.Axis(title='', 
                                                                     titleFontSize = fontSize, labelFontSize = fontSize,
                                                                     labelLimit=1000)),
        y=alt.Y('Seq2:O', sort=ordered_names_filtered, axis=alt.Axis(title='', 
                                                                     titleFontSize = fontSize, labelFontSize = fontSize,
                                                                     labelLimit=1000)),
        color=alt.Color('Hamming Distance:Q').scale(scheme='greys'),
        tooltip=['Seq1', 'Seq2', 'Hamming Distance']
    ).properties(
        title=f'Pairwise {region} Hamming distances'
    ).configure_view(strokeWidth=0
                    ).configure_axis(grid=False, domain=False
                                    ).configure_legend(titleFontSize=fontSize, labelFontSize=fontSize
                                                      ).configure_title(fontSize=fontSize)

    return heatmap


In [6]:
# Make heatmaps of just vaccine strains
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)
    heatmap = plot_triangular_hamming_distances(fasta_file, sequences_to_plot = vaccine_strains, ordered_names = viruses)

    # Save HA1 heatmap as separate object
    if 'HA1' in fasta_file:
        HA1_heatmap = heatmap

    # Show results
    print(f'plotting results from {fasta_file}...')
    heatmap.display()

# Save HA1 heatmap
outfile = os.path.join(resultsdir, 'vaccine_HA1_hamming_distance_heatmap.svg')
HA1_heatmap.save(outfile, dpi = 600)
print(f'HA1 heatmap saved to {outfile}')

plotting results from ../results/2023-2024_H3_library_protein_HA_ectodomain.fasta...


alt.Chart(...)

plotting results from ../results/2023-2024_H3_library_protein_HA1.fasta...


alt.Chart(...)

HA1 heatmap saved to ../results/vaccine_HA1_hamming_distance_heatmap.svg


In [7]:
# Make heatmaps of just library (no vaccine) strains
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)
    heatmap = plot_triangular_hamming_distances(fasta_file, sequences_to_plot = library_strains, ordered_names = viruses)
    # Show results
    print(f'plotting results from {fasta_file}...')
    heatmap.display()

plotting results from ../results/2023-2024_H3_library_protein_HA_ectodomain.fasta...


alt.Chart(...)

plotting results from ../results/2023-2024_H3_library_protein_HA1.fasta...


alt.Chart(...)

The library strain heatmaps are a bit unwieldy, so making histograms of
1) all pairwise Hammming distances
2) all per-strain minimum Hamming distances

In [8]:
def plot_hamming_distance_histograms(fasta_file, sequences_to_plot, ordered_names):
    # Configure
    width = 300
    height = 220
    color_all = color_scheme[0]
    color_min = color_scheme[2]
    fontSize=18

    barSize = 18

    region = fasta_file.split('.fasta')[0].split('_')[-1]
    
    # Load sequences and their names from FASTA
    sequences, names = load_fasta_sequences(fasta_file)

    # Check if sequences are of the same length, which is required for Hamming distance
    seq_len = set(len(seq) for seq in sequences)
    if len(seq_len) != 1:
        raise ValueError("All sequences must be of the same length to compute Hamming distances.")

    # Compute pairwise Hamming distances
    dist_matrix = compute_hamming_distances(sequences)
    
    # Create a DataFrame for the distance matrix using sequence names
    df = pd.DataFrame(dist_matrix, columns=names, index=names)

    # If a subset of sequences is provided, filter the distance matrix
    if sequences_to_plot is not None:
        df = df.loc[sequences_to_plot, sequences_to_plot]

    # Melt the DataFrame to long format for processing
    df_melt = df.reset_index().melt(id_vars='index')
    df_melt.columns = ['Seq1', 'Seq2', 'Hamming Distance']

    # If a custom order is provided, use it to order the axes, keeping only the names in sequences_to_plot
    if ordered_names is not None:
        ordered_names_filtered = [name for name in ordered_names if name in df.columns]
    else:
        ordered_names_filtered = df.columns.tolist()

    # Create a dictionary to map sequence names to their order
    order_dict = {name: idx for idx, name in enumerate(ordered_names_filtered)}

    # Filter for the lower triangle: Keep only rows where Seq1 appears later than Seq2 in the order,
    # and exclude the diagonal (Seq1 == Seq2)
    df_melt = df_melt[
        (df_melt['Seq1'].map(order_dict) > df_melt['Seq2'].map(order_dict)) &
        (df_melt['Seq1'] != df_melt['Seq2'])
    ]
    
    # Histogram 1: All pairwise Hamming distances
    histogram_all = alt.Chart(df_melt, height=height).mark_bar(width=barSize, color = color_all).encode(
        x=alt.X('Hamming Distance:O', 
                axis=alt.Axis(title='Hamming distance',
                              labelAngle=0, labelAlign='center',
                              labelFontSize=fontSize, titleFontSize=fontSize)),
        y=alt.Y('count():Q', axis=alt.Axis(title='count', grid=False, labelFontSize=fontSize, titleFontSize=fontSize)),
    ).properties(
        title=[f'All pairwise {region} Hamming distances', 'among 2023-circulating strains']
    )

    # Histogram 2: Per-strain minimum Hamming distances
    min_distances = df_melt[df_melt['Seq1'] != df_melt['Seq2']].groupby('Seq1')['Hamming Distance'].min().reset_index()
    histogram_min = alt.Chart(min_distances, height=height).mark_bar(width=barSize, color = color_min).encode(
        x=alt.X('Hamming Distance:O', 
                axis=alt.Axis(title='Hamming distance',
                              labelAngle=0, labelAlign='center',
                              labelFontSize=fontSize, titleFontSize=fontSize)),
        y=alt.Y('count(Hamming Distance)',
                axis=alt.Axis(title='count', grid=False, labelFontSize=fontSize, titleFontSize=fontSize)),
        # tooltip=['count()']
    ).properties(
        title=['Per-strain minimum', f'{region} Hamming distances', 'among 2023-circulating strains']
    )

    return histogram_all, histogram_min


In [9]:
# Make histos of just library (no vaccine) strains
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)
    # heatmap = plot_triangular_hamming_distances(fasta_file, sequences_to_plot = library_strains, ordered_names = viruses)
    # heatmap.display()

    hist_all, hist_min = plot_hamming_distance_histograms(fasta_file, sequences_to_plot = library_strains, ordered_names = viruses)
    
    # To visualize
    print(f'plotting results for {fasta_file}...')
    histograms = alt.concat(hist_all, hist_min,)

    if 'HA1' in fasta_file:
        HA1_histograms = histograms
            
    histograms.display()


plotting results for ../results/2023-2024_H3_library_protein_HA_ectodomain.fasta...


alt.ConcatChart(...)

plotting results for ../results/2023-2024_H3_library_protein_HA1.fasta...


alt.ConcatChart(...)

It would also be useful to know the Hamming distances for library strain from a given pivot strain. We'll probably go with A/Massachusetts/18/2022, which is the 2024-2025 cell-based vaccine component. 

In [10]:
def plot_hamming_distance_histogram_from_pivot(fasta_file, sequences_to_plot, ordered_names, pivot_name):
    # Configure
    width=300
    height=220
    color = color_scheme[1]
    fontSize=18
    
    region = fasta_file.split('.fasta')[0].split('_')[-1]
    
    barSize=18
    
    # Load sequences and their names from FASTA
    sequences, names = load_fasta_sequences(fasta_file)

    # Check if sequences are of the same length, which is required for Hamming distance
    seq_len = set(len(seq) for seq in sequences)
    if len(seq_len) != 1:
        raise ValueError("All sequences must be of the same length to compute Hamming distances.")

    # Compute pairwise Hamming distances
    dist_matrix = compute_hamming_distances(sequences)
    
    # Create a DataFrame for the distance matrix using sequence names
    df = pd.DataFrame(dist_matrix, columns=names, index=names)

    # If a subset of sequences is provided, filter the distance matrix
    if sequences_to_plot is not None:
        df = df.loc[sequences_to_plot, sequences_to_plot]

    # Ensure the pivot strain is in the dataset
    if pivot_name not in df.index:
        raise ValueError(f"The pivot strain '{pivot_name}' is not in the provided sequence dataset.")

    # Extract distances for the pivot strain, excluding itself
    pivot_distances = df.loc[pivot_name][df.index != pivot_name]

    # Convert to DataFrame for Altair plotting
    pivot_df = pivot_distances.reset_index()
    pivot_df.columns = ['Seq', 'Hamming Distance']

    # Histogram of counts of sequences with distances from the pivot strain
    histogram_pivot = alt.Chart(pivot_df, height=height).mark_bar(width=barSize, color=color).encode(
        x=alt.X(
            'Hamming Distance:O', 
                axis=alt.Axis(title=['Hamming distance'],
                              labelAngle=0, labelAlign='center',
                              labelFontSize=fontSize, titleFontSize=fontSize)),
        y=alt.Y('count():Q', axis=alt.Axis(title='count', grid=False, labelFontSize=fontSize, titleFontSize=fontSize)),
        tooltip=['count()']
    ).properties(
        title=[f'{region} Hamming distances', 'from reference strain', f'({pivot_name})', 'among 2023-circulating strains']
    )

    return histogram_pivot

In [11]:
# Pivot strain name
pivot_strain = 'A/Massachusetts/18/2022'

# Make histos of just library (no vaccine) strains
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)
    print(f'plotting results for {fasta_file}...')
      
    # Plot the histogram
    hist_pivot = plot_hamming_distance_histogram_from_pivot(fasta_file, 
                                                            sequences_to_plot = library_strains, 
                                                            ordered_names = viruses, 
                                                            pivot_name = pivot_strain)

    # Save HA1 plots separately
    if 'HA1' in fasta_file:
        HA1_hist_pivot = hist_pivot
    
    # Display the histogram
    hist_pivot.display()

plotting results for ../results/2023-2024_H3_library_protein_HA_ectodomain.fasta...


alt.Chart(...)

plotting results for ../results/2023-2024_H3_library_protein_HA1.fasta...


alt.Chart(...)

In [12]:
HA1_plots = (alt.concat(HA1_histograms, HA1_hist_pivot)
             .configure_title(fontSize=18)
             .configure_axis(titleFontSize=18, labelFontSize=18)
            )

# Save
outfile = os.path.join(resultsdir, 'library_HA1_hamming_distance_histograms.svg')
HA1_plots.save(outfile, dpi = 600)
HA1_plots

alt.ConcatChart(...)

## Produce plots of average Hamming distance for each strain, plotted by subclade

In [13]:
# Calculate average Hamming distance per strain
def summarize_hamming_distance(dist_matrix):
    # Convert the distance matrix to a DataFrame for easier processing
    df = pd.DataFrame(dist_matrix)

    # Get reduced df without 0s
    df.replace(0, pd.NA, inplace=True)
    
    # Calculate the average distance for each strain (row)
    mean_distances = df[(df != 0).all(axis=1)].mean(axis=1, skipna=True)
    # Calculate the median distance for each strain (row)
    median_distances = df[(df != 0).all(axis=1)].median(axis=1, skipna=True)    
    # Get the min distance for each strain (row)
    min_distances = df[(df != 0).all(axis=1)].min(axis=1, skipna=True)    
    # Get the max distance for each strain (row)
    max_distances = df[(df != 0).all(axis=1)].max(axis=1, skipna=True)    
    
    return mean_distances, median_distances, min_distances, max_distances


# Main function to handle the plotting
def plot_average_hamming_distances(fasta_file, group_df, sequences_to_plot=None):
    # Load sequences and their names from FASTA
    sequences, names = load_fasta_sequences(fasta_file)

    # If a subset of sequences is provided, filter the distance matrix
    if sequences_to_plot is not None:
        # If there is a filter, filter seq and names lists
        sequences_temp = []
        names_temp = []

        # Iterate through and filter
        for i in list(range(len(names))):
            seq = sequences[i]
            n = names[i]

            if n in sequences_to_plot:
                sequences_temp.append(seq)
                names_temp.append(n)
                
        # Return overwritten lists of sequences and names
        sequences = sequences_temp
        names = names_temp

    # Check if sequences are of the same length, which is required for Hamming distance
    seq_len = set(len(seq) for seq in sequences)
    if len(seq_len) != 1:
        raise ValueError("All sequences must be of the same length to compute Hamming distances.")

    # Compute pairwise Hamming distances
    dist_matrix = compute_hamming_distances(sequences)
    
    # Calculate mean, median, min and max Hamming distance per strain
    mean, median, _min, _max = summarize_hamming_distance(dist_matrix)
    
    # Create a DataFrame of average distances with strain names
    summarized_distances_df = pd.DataFrame({
        'strain': names,
        'Mean Hamming Distance': mean,
        'Median Hamming Distance': median,
        'min': _min,
        'max': _max
    })

    # Merge with subclade information
    merged_df = summarized_distances_df.merge(group_df, on='strain', how='left')

    # Plot with Altair
    scatter = alt.Chart(merged_df).mark_circle(size=60, opacity=0.5).encode(
        x=alt.X('subclade:O', 
                axis = alt.Axis(grid=False, titleFontSize=18, labelFontSize=16,)),  # No specific order unless defined
        y=alt.Y('Mean Hamming Distance:Q', axis = alt.Axis(grid=False, titleFontSize=18, labelFontSize=16,)),
        color=alt.Color('subclade:N', legend=None),  # Color by group
        detail='strain',
        xOffset="jitter:Q",
        tooltip=['strain', 'Mean Hamming Distance', 'subclade']
    ).properties(
        title=f'Average Hamming Distances Per Sequence for {fasta_file}',
        width=600,
        height=300
    ).transform_calculate(
        jitter="sqrt(-4*log(random()))*cos(2*PI*random())" # Generate Gaussian jitter with a Box-Muller transform
    )

    return scatter, merged_df

## Plot Hamming distances for circulating strains by subclade

In [14]:
# Make scatterplots of average Hamming distance for just library (no vaccine) strains
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)
    # Generate the plot
    chart, df = plot_average_hamming_distances(fasta_file, strain_metadata, sequences_to_plot=library_strains)
    chart.display()

    print('Median distances across strains range from... ', df['Median Hamming Distance'].min(), df['Median Hamming Distance'].max())
    print('Minimum distances across strains range from... ', df['min'].min(), df['min'].max())

alt.Chart(...)

Median distances across strains range from...  2.0 11.0
Minimum distances across strains range from...  1.0 3.0


alt.Chart(...)

Median distances across strains range from...  2.0 10.0
Minimum distances across strains range from...  1.0 3.0


## Plot Hamming distances for vaccine strains by subclade

In [15]:
# Make scatterplots of average Hamming distance for just vaccine strains
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)
    # Generate the plot
    chart, df = plot_average_hamming_distances(fasta_file, strain_metadata, sequences_to_plot=vaccine_strains)
    chart.display()

    print('Median distances across strains range from... ', df['Median Hamming Distance'].min(), df['Median Hamming Distance'].max())
    print('Minimum distances across strains range from... ', df['min'].min(), df['min'].max())

alt.Chart(...)

Median distances across strains range from...  13.5 23.0
Minimum distances across strains range from...  2.0 6.0


alt.Chart(...)

Median distances across strains range from...  11.5 21.5
Minimum distances across strains range from...  2.0 4.0


## Produce plots with median distance (with underlying scatter?) by epitope

In [16]:
# Antigenic regions 
# These are epitopes based on Broecker et al. 2018:
# https://journals.asm.org/doi/10.1128/jvi.01100-18

epitope_A = list(range(122,147))
epitope_B = list(range(155,161)) + list(range(186,199))
epitope_C = list(range(44,55)) + list(range(273,281))
epitope_D = list(range(166,182)) + list(range(201,220))
epitope_E = list(range(62,66)) + list(range(78,95)) + list(range(260,266))

region_data = {
    'antigenic_site': ['A', 'B', 'C', 'D', 'E'],
    'region': [epitope_A, epitope_B, epitope_C, epitope_D, epitope_E]
}

In [17]:
# Function to calculate Hamming distance between two sequences for specific regions
def hamming_distance_in_regions(seq1, seq2, positions):
    return sum(seq1[i] != seq2[i] for i in positions)

# Function to calculate Hamming distance for regions outside the specified positions
def hamming_distance_outside_regions(seq1, seq2, positions, seq_len):
    other_positions = set(range(seq_len)) - set(positions)
    return sum(seq1[i] != seq2[i] for i in other_positions)

# Compute pairwise Hamming distances for specific regions
def compute_hamming_distances_by_region(sequences, region_positions, seq_len):
    n = len(sequences)
    
    dist_in_regions = np.zeros((n, n))
    
    for i in range(n):
        for j in range(i, n):
            dist_in = hamming_distance_in_regions(sequences[i], sequences[j], region_positions)
            dist_in_regions[i, j] = dist_in_regions[j, i] = dist_in
    
    return dist_in_regions

# Compute pairwise Hamming distances outside of all the specified regions
def compute_hamming_distances_outside_all_regions(sequences, all_region_positions, seq_len):
    n = len(sequences)
    
    dist_outside_regions = np.zeros((n, n))
    
    other_positions = set(range(seq_len)) - set(all_region_positions)
    
    for i in range(n):
        for j in range(i, n):
            dist_out = hamming_distance_in_regions(sequences[i], sequences[j], other_positions)
            dist_outside_regions[i, j] = dist_outside_regions[j, i] = dist_out
    
    return dist_outside_regions

# Function to plot pairwise Hamming distances for each region and outside those regions
def plot_pairwise_distances_by_region(fasta_file, region_data):
    # Load sequences and their names from FASTA
    sequences, names = load_fasta_sequences(fasta_file)

    # Ensure all sequences are of the same length
    seq_len = set(len(seq) for seq in sequences)
    if len(seq_len) != 1:
        raise ValueError("All sequences must be of the same length to compute Hamming distances.")
    seq_len = len(sequences[0])
    
    # Prepare to store all distances
    distances = []
    all_region_positions = []
    
    # Compute pairwise Hamming distances for each specified region
    for site, region_positions in zip(region_data['antigenic_site'], region_data['region']):
        all_region_positions.extend(region_positions)
        
        dist_in_regions = compute_hamming_distances_by_region(sequences, region_positions, seq_len)
        
        # Flatten the matrix and prepare data for plotting
        dist_in_flat = dist_in_regions[np.triu_indices_from(dist_in_regions, k=1)]
        
        # Add region label for each distance
        distances.append(pd.DataFrame({
            'Region': [site] * len(dist_in_flat),
            'Hamming Distance': dist_in_flat
        }))
    
    # Compute pairwise Hamming distances for sites outside all specified regions
    dist_outside_regions = compute_hamming_distances_outside_all_regions(sequences, all_region_positions, seq_len)
    dist_outside_flat = dist_outside_regions[np.triu_indices_from(dist_outside_regions, k=1)]
    
    # Add 'Other' region for the distances outside specified regions
    distances.append(pd.DataFrame({
        'Region': ['Other'] * len(dist_outside_flat),
        'Hamming Distance': dist_outside_flat
    }))
    
    # Concatenate all regions into a single DataFrame
    df = pd.concat(distances)

    # Create scatter plot with jitter and overlaid boxplot for each region
    scatter = alt.Chart(df).mark_circle(size=60, opacity=0.3, filled=False).encode(
        x=alt.X('Region:N', title='Antigenic site',
               axis = alt.Axis(grid=False, titleFontSize=18, labelFontSize=16, labelAngle = 0)),
        y=alt.Y('Hamming Distance:Q', title='Hamming distance',
               axis = alt.Axis(grid=False, titleFontSize=18, labelFontSize=16,)),
        color=alt.Color('Region:N', legend=None).scale(scheme='set2'),
        tooltip=['Region', 'Hamming Distance'],
        xOffset='jitter:Q'
    ).properties(
        width=400,
        height=300
    ).transform_calculate(
        jitter="sqrt(-4*log(random()))*cos(2*PI*random())" # Generate Gaussian jitter with a Box-Muller transform
    )

    # Create boxplot
    boxplot = alt.Chart(df).mark_boxplot(extent="min-max", opacity = 0.8, size = 20, color = 'white').encode(
        x=alt.X('Region:N'),
        y=alt.Y('Hamming Distance:Q', title='Hamming distance'),
        stroke = alt.value('black'),
        strokeWidth=alt.value(2)
    )

    # Combine scatter and boxplot
    chart = alt.layer(scatter, boxplot
                     ).properties(title=f'Pair-wise Hamming distances for {fasta_file}'
                                 ).configure_title(fontSize = 18)

    return chart

In [18]:
# Flatten the region positions into a single list
region_positions = [pos for region in region_data['region'] for pos in region]

# Show for both ectodomain and HA1
for file in protein_sequences:
    fasta_file = os.path.join(resultsdir, file)

    # Generate the plot
    chart = plot_pairwise_distances_by_region(fasta_file, region_data)
    chart.display()


alt.LayerChart(...)

alt.LayerChart(...)

## Code graveyard

In [19]:
# # Define strain metadata
# strain_metadatafile = os.path.join(datadir, 'strain_metadata.csv')
# strain_metadata = (pd.read_csv(strain_metadatafile, sep = ';')
#                    .drop(columns = 'index')
#                    .rename(columns = {'seqName': 'strain'}))

# # Replace values in strain metadata with strings matching the plotted strain names
# virus_dict = {'A/Texas/50/2012_X-223A_13/252': 'A/Texas/50/2012_X-223A_(13/252)',
#               'A/Hong_Kong/4801/2014_15/192': 'A/Hong_Kong/4801/2014_(15/192)'}

# strain_metadata['strain'] = strain_metadata['strain'].replace(virus_dict)

In [20]:
# # Function to calculate Hamming distance between two sequences
# def hamming_distance(seq1, seq2, region=None):
#     if region is None:  # If no region is specified, compare the whole sequence
#         return sum(c1 != c2 for c1, c2 in zip(seq1, seq2))
#     else:
#         # Compare only specified region
#         start, end = region
#         return sum(c1 != c2 for c1, c2 in zip(seq1[start:end], seq2[start:end]))

# # Load protein sequences and their names from FASTA file
# def load_fasta_sequences(fasta_file):
#     sequences = []
#     names = []
#     for record in SeqIO.parse(fasta_file, "fasta"):
#         sequences.append(str(record.seq))
#         names.append(record.id)  # Use record.id for sequence name (header)
#     return sequences, names

# # Compute pairwise Hamming distances
# def compute_hamming_distances(sequences):
#     n = len(sequences)
#     dist_matrix = np.zeros((n, n))
    
#     for i in range(n):
#         for j in range(i, n):
#             dist = hamming_distance(sequences[i], sequences[j])
#             dist_matrix[i, j] = dist
#             dist_matrix[j, i] = dist
            
#     return dist_matrix

# # Main function to handle the plotting
# def plot_hamming_distances(fasta_file, sequences_to_plot, ordered_names):
#     # Load sequences and their names from FASTA
#     sequences, names = load_fasta_sequences(fasta_file)

#     # Check if sequences are of the same length, which is required for Hamming distance
#     seq_len = set(len(seq) for seq in sequences)
#     if len(seq_len) != 1:
#         raise ValueError("All sequences must be of the same length to compute Hamming distances.")

#     # Compute pairwise Hamming distances
#     dist_matrix = compute_hamming_distances(sequences)
    
#     # Create a DataFrame for the distance matrix using sequence names
#     df = pd.DataFrame(dist_matrix, columns=names, index=names)

#     # Melt the DataFrame to long format for Altair plotting
#     df_melt = df.reset_index().melt(id_vars='index')
#     df_melt.columns = ['Seq1', 'Seq2', 'Hamming Distance']

#     # If a subset of sequences is provided, filter the distance matrix
#     if sequences_to_plot is not None:
#         df = df.loc[sequences_to_plot, sequences_to_plot]

#     # Melt the DataFrame to long format for Altair plotting
#     df_melt = df.reset_index().melt(id_vars='index')
#     df_melt.columns = ['Seq1', 'Seq2', 'Hamming Distance']

#     # If a custom order is provided, use it to order the axes, keeping only the names in sequences_to_plot
#     if ordered_names is not None:
#         ordered_names_filtered = [name for name in ordered_names if name in df.columns]
#     else:
#         ordered_names_filtered = df.columns.tolist()

#     # Plot with Altair
#     heatmap = alt.Chart(df_melt).mark_rect().encode(
#         x=alt.X('Seq1:O', sort=ordered_names_filtered, axis = alt.Axis(title='virus')),
#         y=alt.Y('Seq2:O', sort=ordered_names_filtered, axis = alt.Axis(title='virus')),
#         color='Hamming Distance:Q',
#         tooltip=['Seq1', 'Seq2', 'Hamming Distance']
#     ).properties(
#         title=f'Pairwise Hamming Distances for {fasta_file}'
#     )

#     return heatmap
#     return heatmap
